In [1]:
from skimage.transform import rescale
from skimage.io import imread, imsave
from tqdm import tqdm
import os
import numpy as np

In [2]:
path_20x = r"\\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\20x Native"
path_40x = r"\\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\40x Native"
path_int = r"\\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\Interpolations"
Ls_and_Rs = os.listdir(path_int)

In [3]:
def downsample(img, msk, order):
    img = rescale(img, 0.5, order=order, channel_axis=2)
    msk = rescale(msk, 0.5, order=0)
    if order > 0:
        img = img * 255
    return img.astype(np.uint8), msk.astype(np.uint16)

def upsample(img, msk, order):
    img = rescale(img, 2, order=order, channel_axis=2)
    msk = rescale(msk, 2, order=0)
    if order > 0:
        img = img * 255
    return img.astype(np.uint8), msk.astype(np.uint16)

In [4]:
# First the L's which start from 40x Native
Ls = [L for L in Ls_and_Rs if 'L' in L]
datasets = os.listdir(path_40x)
for dataset in datasets:
    path_imgs_in = os.path.join(os.path.join(path_40x, dataset), 'images')
    path_msks_in = os.path.join(os.path.join(path_40x, dataset), 'masks')
    for L in tqdm(Ls):
        path_imgs_out = os.path.join(os.path.join(os.path.join(path_int, L), dataset), 'images')
        path_msks_out = os.path.join(os.path.join(os.path.join(path_int, L), dataset), 'masks')
        for filename in os.listdir(path_imgs_in):
            if '.tif' in filename:
                img = imread(os.path.join(path_imgs_in, filename))
                msk = imread(os.path.join(path_msks_in, filename))
                if '__' in L:  # Leave as Native
                    pass
                elif '_' in L:  # Downsample only
                    order = eval(L[-2])
                    img, msk = downsample(img, msk, order)
                else:  # Downsample then Upsample
                    order_down, order_up = eval(L[-2]), eval(L[-1])
                    img, msk = downsample(img, msk, order_down)
                    img, msk = upsample(img, msk, order_up)
                imsave(os.path.join(path_imgs_out, filename), img)
                imsave(os.path.join(path_msks_out, filename), msk, check_contrast=False)

100%|██████████| 13/13 [01:53<00:00,  8.71s/it]


In [5]:
# Next the R's which start from 20x Native
Rs = [R for R in Ls_and_Rs if 'R' in R]
datasets = os.listdir(path_20x)
for dataset in datasets:  # just JHU
    path_imgs_in = os.path.join(os.path.join(path_20x, dataset), 'images')
    path_msks_in = os.path.join(os.path.join(path_20x, dataset), 'masks')
    for R in tqdm(Rs):
        path_imgs_out = os.path.join(os.path.join(os.path.join(path_int, R), dataset), 'images')
        path_msks_out = os.path.join(os.path.join(os.path.join(path_int, R), dataset), 'masks')
        for filename in os.listdir(path_imgs_in):
            if '.tif' in filename:
                img = imread(os.path.join(path_imgs_in, filename))
                msk = imread(os.path.join(path_msks_in, filename))
                if '__' in R:  # Leave as Native
                    pass
                elif '_' in R:  # Upsample only
                    order = eval(R[-2])
                    img, msk = upsample(img, msk, order)
                else:  # Upsample then Downsample
                    order_up, order_down = eval(R[-2]), eval(R[-1])
                    img, msk = upsample(img, msk, order_up)
                    img, msk = downsample(img, msk, order_down)
                imsave(os.path.join(path_imgs_out, filename), img)
                imsave(os.path.join(path_msks_out, filename), msk, check_contrast=False)

100%|██████████| 13/13 [03:22<00:00, 15.56s/it]
